In the last section with Postman, we signed up with openweathermap API and obtained an `APPID`. We are going to send similiar `GET` queries to the API to retrieve weather data but using Python's requests library.

In [141]:

pd.read_csv(CSV_FILEPATH)

,id,lat,lon,city,state
0,0,41,80,"""Youngstown""",OH
1,1,42,97,"""Yankton""",SD
2,2,46,120,"""Yakima""",WA
3,3,42,71,"""Worcester""",MA
4,4,43,89,"""Wisconsin Dells""",WI
...,...,...,...,...,...
123,123,39,119,"""Reno""",NV
124,124,50,104,"""Regina""",SA
125,125,40,122,"""Red Bluff""",CA
126,126,40,75,"""Reading""",PA


In [138]:
df.columns

Index(['Unnamed: 0', 'lat', 'lon', 'city', 'state'], dtype='object')

In [139]:
df = df.rename({'Unnamed: 0':'id' }, axis = 1)

In [140]:
df.to_csv('/home/asimbanskota/t81_577_data_science/weekly_materials/week5/files/city.csv', index = False)

Requests library comes with the standard distribution of Anaconda. Otherwise, you can install it using either pip or conda install.

```shell
conda install requests
```

or 

```shell
pip install requests
```


In [89]:
import requests

### A little divergence from requests

APIs authenticate an user using various mechanisms. While weathermapapi authenticates using `APPID` for its free-users, other APIs might use `public-private-keys` and `secret-access-keys` as in AWS, or through conventional `username` or `password` credentials. Obviously, we want to store and use them in a secure manner. 

You can simply copy paste your `APPID` when prompted using `getpass` module as follows if you don't mind copying the key everytime you run your script or notebook. 

In [90]:
import getpass
APPID = getpass.getpass(prompt='APPID: ', stream=None)

APPID: ········


Alternatively, I stored the APPID as an environment variable. To do that, you need to activate your environment and run the following command and reactivate conda environment:

```bash
conda env config vars set APPID=your_appid
```

To retrive the stored APPID environment variable, run the following commands:

In [91]:
import os
APPID = os.getenv("APPID")

### Back to making `GET` request to API

Lets first make a simple GET request to openweathermap API by calling get() with the following URL:

In [92]:
url = 'https://api.openweathermap.org/data/2.5/weather?q=Saint Louis&APPID={}'.format(APPID)

response = requests.get(url)

To check whether the server returned any data, you can call `status_code` attribute of the response:

In [93]:
response.status_code

200

We learnt in the last section that the status 200 means the request was successful. Requests has a built-in status code lookup object for easy handling of the response appropriately. An exception can be raised if the response is not `ok` using `raise_for_status()` method. 

In [99]:
url_e = 'https://api.openweathermap.org/data/2.5/weather?q=Saint Louis&APPID={}'.format('74892b4hjb4jhb24bhj')

In [100]:
from requests.exceptions import HTTPError

try:
    response = requests.get(url_e)
    response.raise_for_status()
except requests.exceptions.HTTPError as errh:
    print ("An HTTP Error occurred: {}".format(errh))
except requests.exceptions.ConnectionError as errc:
    print ("An Error Connecting to the API occurred:{}".format(errc))
except requests.exceptions.Timeout as errt:
    print("A Timeout Error occurred: {}".format(errt))
except requests.exceptions.RequestException as err:
    print("An Unknown Error occurred: {}".format(err))
except Exception as err:
    print('Error occurred: {}'.format(err)) 
else:
    print('Success!')

An HTTP Error occurred: 401 Client Error: Unauthorized for url: https://api.openweathermap.org/data/2.5/weather?q=Saint%20Louis&APPID=74892b4hjb4jhb24bhj


If the status code indicates a successful request, the program will proceed without exception being raised. In the above case, the status code was 401, hence an HTTPError exception was raised caused by a wrong APPID.

### Passing Parameters In URLs

In the examples we are working,we used the complete URL constructed by hand i.e. manually typing the query parameter for city and APPID after base_url (https://api.openweathermap.org/data/2.5/weather?). This is cumbersome and prone to error. Requests allows you to provide these arguments as a dictionary, using the params keyword argument.



In [96]:
params = {'q': 'Saint Louis', 'APPID': APPID}
response = requests.get("https://api.openweathermap.org/data/2.5/weather?", params=params)

you can verify that the request used the same url as before by typing:

```python
reponse.url
```

You can keep adding other parameters as needed. For example, to retrive temperature unit in Farenheit, you can add key,value pair `units` and `imperial.

In [97]:
params = {'q': 'Saint Louis', 'units': 'imperial', 'APPID': APPID}
response = requests.get("https://api.openweathermap.org/data/2.5/weather?", params=params)

### Reading contents

You can access the content of the reponse by using `response.content` for binary encoded data or reponse.text for text data. W knew from the previous exercise that the API returns reponse in `JSON` format unless specified otherwise. Request reponse has builtin `json` method to decode JSON contents.

In [98]:
response.json()

{'coord': {'lon': -90.2, 'lat': 38.63},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 52.68,
  'feels_like': 44.76,
  'temp_min': 46,
  'temp_max': 59,
  'pressure': 1014,
  'humidity': 71},
 'visibility': 16093,
 'wind': {'speed': 11.41, 'deg': 180, 'gust': 23.04},
 'clouds': {'all': 90},
 'dt': 1581276715,
 'sys': {'type': 1,
  'id': 3689,
  'country': 'US',
  'sunrise': 1581253152,
  'sunset': 1581291055},
 'timezone': -21600,
 'id': 4407066,
 'name': 'St. Louis',
 'cod': 200}